In [1]:
from pydantic_ai import Agent, RunContext, Tool
from pydantic_ai.models.ollama import OllamaModel # type: ignore
from pydantic import BaseModel, Field
from typing import Dict, List, Optional
from datetime import date

from dataclasses import dataclass

import nest_asyncio
nest_asyncio.apply()

In [2]:
model = OllamaModel(
    model_name='qwen2.5:7b',  
    base_url='http://localhost:11434/v1/',  
)

In [3]:
# Define the structured response schema; The agent should respond to the client in this way
class ResponseModel(BaseModel):
    """Structured response."""
    response: str
    question_topic: str 
    needs_escalation: bool = False
    follow_up_required: bool =False
    sentiment: str = Field(description="Customer sentiment analysis") 

In [4]:
@dataclass
class MyDeps:
    clients_name: str

In [5]:

# Agent with structured output and dependencies
agent4 = Agent(
    model=model,
    retries=2,
    result_type=ResponseModel,
    deps_type=MyDeps,
)

In [6]:
#so how do i make the agent aware fo the context that is knwon during the conversation? by adding RunContext.

@agent4.system_prompt
async def dynamic_system_prompt(ctx: RunContext[MyDeps]) -> str:
    """Generates a system prompt using dependency data."""
    clients_name = ctx.deps.clients_name
    print(f"Dynamic prompt generated for client : {clients_name}")  # Debugging
    return (
        f"You are an intelligent support agent. "
        f"Analyze queries carefully and provide brief and structured responses. "
        f"Ensure every response includes the clients name: '{clients_name}'. "
        "Ask clarifying questions if necessary."
    )

In [7]:
# im setting a dependency by provider the agent with an acccount name. this is required becasue it is registered with the agent.
current_client = MyDeps(clients_name="Angela")

In [8]:
# Query the agent for account information
response = agent4.run_sync(user_prompt="I am seeing alot of bad data coming in. did something change recently?", 
                           deps=current_client)
# Output the response
print(response.data.model_dump_json(indent=2))

/var/folders/nq/y2lyg7p15txfm5ksrw6f90340000gn/T/ipykernel_29176/1775348045.py:2: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  response = agent4.run_sync(user_prompt="I am seeing alot of bad data coming in. did something change recently?",


Dynamic prompt generated for client : Angela
{
  "response": "It sounds like you're experiencing issues with the quality of incoming data, Angela. Could you provide more details about what specifically has changed or how this impact is being observed? This will help me understand better and assist you effectively.",
  "question_topic": "Data Quality Issues",
  "needs_escalation": false,
  "follow_up_required": true,
  "sentiment": "Concerned"
}


In [10]:
# Query the agent for account information
response2 = agent4.run_sync(user_prompt="We are not getting any sales from these leads we have bought.This is BS. what can we do?", 
                           deps=current_client)
# Output the response
print(response2.data.model_dump_json(indent=2))

Dynamic prompt generated for client : Angela
{
  "response": "Let's review the specifics of these leads to identify areas for improvement in the follow-up process. I can help tailor strategies that might increase your chances of converting these leads.",
  "question_topic": "sales lead management",
  "needs_escalation": false,
  "follow_up_required": true,
  "sentiment": "frustrated"
}
